<a href="https://colab.research.google.com/github/GeomaticsCaminosUPM/SeismicBuildingExposure/blob/main/examples/example_irregularity_norms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Irregularity module example

This example explains the parameters of different norms that can be computed

Install libraries if using colab

In [13]:
!pip install geopandas
!pip install git+https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure.git
!pip install folium matplotlib mapclassify

In [17]:
import SeismicBuildingExposure.irregularity
import geopandas as gpd

Documentation of every function can be found using `help()` or under https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure

In [18]:
help(SeismicBuildingExposure.irregularity.mexico_NTC_df)

Help on function mexico_NTC_df in module SeismicBuildingExposure.irregularity:

mexico_NTC_df(geoms: geopandas.geodataframe.GeoDataFrame, max_slenderness=4) -> pandas.core.frame.DataFrame



## 1. Load data

Load the footprints geodataframe in `.shp` `.gpkg` `.geojson` format using `geopandas.read_file()`

Download an example footprints file if needed

In [19]:
!curl -L -o footprints.gpkg https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure/raw/refs/heads/main/examples/footprints.gpkg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100  364k  100  364k    0     0   481k      0 --:--:-- --:--:-- --:--:--  481k


Load the footprints geodataframe in `.shp` `.gpkg` `.geojson` format using `geopandas.read_file()`

In [20]:
footprints_gdf = gpd.read_file('footprints.gpkg')

- All geometries should be single part and polygons.

    - Multi part geometries (MultiPolygon) would mean two buildings are contained in the same footprint geometry which should not happen.

    - Footprints must be polygons, not linestrings.

Check if all geometries are `Polygon`

In [21]:
if any(footprints_gdf.geometry.type == 'MultiPolygon'):
    print("There are multiplart geometries.")

if any(footprints_gdf.geometry.type.str.contains('Polygon') == False):
    print("Some geometries are not Polygon")

Explode multipart geometries into single parts if needed


Note: The row number will be reset. Maybe you do not know how to match the footprints with your data. An id column is a good idea to solve this issue.

In [22]:
footprints_gdf = footprints_gdf.explode().reset_index() # The new index column is the row number in the origninal dataframe
footprints_gdf

,index,qc_id,id,hora encue,encuestado,num habita,ano constr,num pisos,alt aprox,num sotano,...,conex mu_4,cimentacio,caracter s,entorno ed,separacion,separaci_1,separaci_2,irregul pl,Edif_Agrup,geometry
0,0,0.0,7.0,5/22/2024 11:42:49,Ing. Omar Gilberto Flores Belteton,None,1980,7,3,0,...,Buena,Zapata aislada con viga de atado,C,None,Mas de 3x numero pisos (cm),Mas de 3x numero pisos (cm),Topan,Regular,NaN,"POLYGON Z ((-90.51386 14.64293 0, -90.5139 14...."
1,1,1.0,8.0,5/28/2024 13:19:27,Ing. Carlos Jose Gamboa Cante,None,None,2,3,1,...,None,Zapata aislada con viga de atado,C,Zona de relleno,Topan,None,Topan,Regular,NaN,"POLYGON Z ((-90.51477 14.64314 0, -90.51483 14..."
2,2,2.0,9.0,5/23/2024 11:41:08,Ing. Carlos Jose Gamboa Cante,None,None,6,3,0,...,Buena,Cimiento corrido,C,Zona de relleno,Topan,Topan,Topan,Regular,NaN,"POLYGON Z ((-90.51513 14.64308 0, -90.51516 14..."
3,3,3.0,10.0,5/23/2024 11:45:15,Ing. Carlos Jose Gamboa Cante,None,None,3,3,0,...,None,Cimiento corrido,C,Zona de relleno,None,Topan,Topan,Regular,NaN,"POLYGON Z ((-90.51533 14.64311 0, -90.51538 14..."
4,4,4.0,53.0,5/23/2024 11:49:48,Ing. Carlos Jose Gamboa Cante,None,None,1,3,0,...,None,Cimiento corrido,C,Zona de relleno,None,Menos de 3x numero pisos (cm),None,None,NaN,"POLYGON Z ((-90.51437 14.64267 0, -90.51436 14..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,431,NaN,1000020.0,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,1.0,"POLYGON Z ((-90.51218 14.64711 0, -90.51218 14..."
432,432,NaN,1000021.0,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,NaN,"POLYGON Z ((-90.51227 14.64726 0, -90.51214 14..."
433,433,NaN,1000024.0,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,NaN,"POLYGON Z ((-90.51426 14.6473 0, -90.51425 14...."
434,434,NaN,1000001.0,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,NaN,"POLYGON Z ((-90.5143 14.64326 0, -90.51423 14...."


## 2. Eurocode 8

Irregularity is measured following the [Eurocode 8 standard](https://www.phd.eng.br/wp-content/uploads/2015/02/en.1998.1.2004.pdf).

The calculated parameters are:

- **Excentricity Ratio**:  
  $
  \text{excentricity\_ratio} = \frac{\text{excentricity}}{\text{torsional radius}}
  $  
  This considers the worst-case value across all possible directions.

- **Radius Ratio**:  
  $
  \text{radius\_ratio} = \frac{\text{torsional radius}}{\text{radius of gyration}}
  $


- **Slenderness**:  
  Typically calculated as $ \frac{L_1}{L_2} $, where $ L_1 $ and $ L_2 $ are the sides of the footprint.  
  Since polygonal shapes may not clearly resemble rectangles, we use:  
  $
  \sqrt{\frac{I_1}{I_2}}
  $  
  where $ I_1 $ and $ I_2 $ are the principal values of the inertia tensor.

- **Compactness**:  
  $
  \text{compactness} = \frac{\text{area of polygon (with all holes filled)}}{\text{area of convex hull}}
  $

The function `eurocode_8_df` returns the **weakest direction** as an angle (in degrees) with respect to the **north (in UTM coordinates)**.

### Definitions:

- **Excentricity**: Distance between the center of mass (centroid of the polygon) and the center of stiffness (centroid of the boundary).
- **Torsional Radius**:  
  $
  \sqrt{\frac{I_t}{I_j}}
  $  
  where:
  - $ I_t $: Inertia in the Z-direction through the center of stiffness.  
  - $ I_j $: Inertia through the center of mass along the axis perpendicular to the calculation direction.

- **Radius of Gyration**:  
  $
  \sqrt{\frac{I_0}{\text{area}}}
  $  
  where:
  - $ I_0 $: Inertia in the Z-direction through the center of mass.

### Eurocode 8 Parameter Limits:

| Parameter             | Limit        |
|----------------------|--------------|
| Excentricity Ratio    | < 0.3        |
| Radius Ratio          | < 1.0        |
| Slenderness           | < 4.0        |
| Compactness           | > 0.95       |



In [23]:
result = SeismicBuildingExposure.irregularity.eurocode_8_df(footprints_gdf)
result

,excentricity_ratio,radius_ratio,slenderness,compactness,angle_excentricity,angle_slenderness
0,0.538997,0.096016,1.593799,0.914525,-55.177478,-21.161538
1,0.322958,0.088353,1.151912,0.974776,-28.854943,-34.426902
2,0.317324,0.106617,1.482460,0.999997,-67.106806,-5.481166
3,0.014569,0.104228,1.667609,0.998978,-85.038906,-7.614403
4,0.006168,0.221249,2.371793,1.000000,33.752556,-83.545271
...,...,...,...,...,...,...
431,0.343228,0.286508,2.446977,0.811570,-34.731937,-24.512346
432,0.025733,0.243583,1.296860,0.997610,-4.270337,-89.932899
433,0.005091,0.201392,1.829355,1.000000,-75.182543,-1.711974
434,0.003936,0.131438,1.991669,0.998113,-62.801053,-7.841958


Save relevant columns of the result to the `footprints_gdf` variable if needed

In [24]:
footprints_gdf['excentricity_ratio_EC8'] = result['excentricity_ratio']
footprints_gdf['radius_ratio_EC8'] = result['radius_ratio']
footprints_gdf['slenderness_EC8'] = result['slenderness']
footprints_gdf['compactness_EC8'] = result['compactness']

Plot a map of building footprints that meet the Eurocode 8 excentricity ratio criterion.

In [25]:
m=footprints_gdf.loc[footprints_gdf['excentricity_ratio_EC8'] <= 0.3].explore(color='green')
m=footprints_gdf.loc[footprints_gdf['excentricity_ratio_EC8'] > 0.3].explore(m=m,color='red')
m

## 3. Costa Rica Código Sísmico Norm

Irregularity is measured following the [Seismic Code of Costa Rica](https://www.codigosismico.or.cr/).

The calculated parameter is:

- **Excentricity Ratio**:  
  $
  \text{excentricity\_ratio} = \frac{\text{excentricity}}{\text{dimension}}
  $  
  considering the **weakest possible direction**.

The function `codigo_sismico_costa_rica_df` returns the **weakest direction** as an angle (in degrees) with respect to the **north (in UTM coordinates)**.

### Definitions:

- **Excentricity**:  
  The distance between the **center of mass** (centroid of the polygon) and the **center of stiffness** (centroid of the boundary).

- **Dimension**:  
  The length of the shape in the considered direction. For rectangles, this is straightforward, but for an arbitrary polygon it is computed as:  
  $
  \text{dimension} = \sqrt{\text{area} \cdot \sqrt{\frac{I_i}{I_j}}}
  $  
  where:
  - $ I_i $: Inertia in the considered direction (through the center of mass).
  - $ I_j $: Inertia in the perpendicular direction (also through the center of mass).

### Código Sísmico de Costa Rica – Parameter Limits:

| Parameter            | Limit                        | Irregularity Level |
|---------------------|------------------------------|--------------------|
| Excentricity Ratio   | $< 0.05$                      | Regular             |
| Excentricity Ratio   | $0.05 < \frac{e}{d} < 0.25$   | Moderate            |
| Excentricity Ratio   | $> 0.25$                      | High                |



In [26]:
result = SeismicBuildingExposure.irregularity.codigo_sismico_costa_rica_df(footprints_gdf)
result

,excentricity_ratio,angle
0,0.027973,-57.923200
1,0.012165,-29.317338
2,0.017365,-69.754050
3,0.000831,-87.997248
4,0.000836,24.629994
...,...,...
431,0.062561,-43.577421
432,0.002959,-3.887172
433,0.000597,-79.078209
434,0.000307,-68.937520


Save relevant variables to `footprints_gdf`

In [27]:
footprints_gdf['excentricity_ratio_CR'] = result['excentricity_ratio']

Plot a map of building footprints that meet the Costa Rica excentricity ratio criterion.

In [28]:
m=footprints_gdf.loc[footprints_gdf['excentricity_ratio_CR'] <= 0.05].explore(color='green')
m=footprints_gdf.loc[(footprints_gdf['excentricity_ratio_CR'] > 0.05) & (footprints_gdf['excentricity_ratio_CR'] < 0.25)].explore(m=m,color='yellow')
m=footprints_gdf.loc[footprints_gdf['excentricity_ratio_CR'] > 0.25].explore(m=m,color='red')
m

C:\Users\Miguel\AppData\Local\Temp\ipykernel_12252\55075204.py:3: UserWarning: The GeoSeries you are attempting to plot is composed of empty geometries. Nothing has been displayed.
  m=footprints_gdf.loc[footprints_gdf['excentricity_ratio_CR'] > 0.25].explore(m=m,color='red')


## 4. Mexico NTC norm

Irregularity is measured following the [Mexico NTC norm]().

The calculated parameters are:

- **Setback Ratio**:  
  $
  \text{setback\_ratio} = \frac{\text{setback length}}{\text{side length}}
  $  
  considering the **worst of the two directions** and the **worst of all setbacks**.

- **Hole Ratio**:  
  $
  \text{holes\_ratio} = \frac{\text{hole width}}{\text{side length}}
  $  
  considering the **worst of the two directions** and the **worst of all holes**.

### Definitions:

- **Length of Side**:  
  The footprint is circumscribed in the **smallest possible rectangle**, with sides aligned to the **principal axes of the inertia tensor** of the footprint.  
  The *side length* refers to the side of this rectangle along either of the principal directions.

  For the **holes ratio**, we consider the **principal axes of each hole shape**, and measure the side length as a line passing through the **center of mass of the hole** in each principal direction.

- **Length of Setback**:  
  Setbacks are defined as the **polygons formed by the difference between the convex hull and the footprint (with holes filled)**.  
  These setback polygons are also circumscribed in a rectangle whose sides are aligned with the **principal directions of the inertia tensor of the footprint**.  
  The *setback length* is the side of this rectangle in one of the two principal directions.  
  In the **setback ratio**, both the *setback length* and the *side length* must be taken in the **same direction** (parallel).

- **Hole Width**:  
  Each hole is circumscribed in a rectangle, with sides aligned to the **principal directions of the hole’s inertia tensor**.  
  The *hole width* is the length of this rectangle in each principal direction.

- **Max Slenderness**: In the context of the **setback ratio**, very thin irregularities caused by concave angles close to 180º comming from imperfections in the footprint,  can lead to a circumscribed rectangle with a disproportionately large side, even though such features may not represent a true setback. **Max slenderness** is the maximum slenderness of **setback circunscribed rectangles**. 

### Mexico NTC – Parameter Limits:

| Parameter        | Limit |
|------------------|--------|
| Setback Ratio     | $< 0.4$ |
| Hole Ratio        | $< 0.4$ |


In [29]:
result = SeismicBuildingExposure.irregularity.mexico_NTC_df(footprints_gdf, max_slenderness=4)
result

,setback_ratio,hole_ratio
0,0.523823,0.000000
1,0.607479,0.000000
2,0.000000,0.523573
3,0.000000,0.000000
4,0.000000,0.000000
...,...,...
431,0.534098,0.000000
432,0.000000,0.000000
433,0.000000,0.000000
434,0.000000,0.000000


Save relevant variables

In [30]:
footprints_gdf['setback_ratio_NTC'] = result['setback_ratio']
footprints_gdf['hole_ratio_NTC'] = result['hole_ratio']

Plot a map of building footprints that meet the Mexico NTC setback ratio criterion.

In [31]:
m=footprints_gdf.loc[footprints_gdf['setback_ratio_NTC'] <= 0.4].explore(color='green')
m=footprints_gdf.loc[footprints_gdf['setback_ratio_NTC'] > 0.4].explore(m=m,color='red')
m

## 5. Save results

In [ ]:
footprints_gdf.to_file('path/to/save/file.gpkg')